# Sorting Notebook

This notebook will download and sort electrophysiology collected using an Intan headstage, in the .rhd format. 

The data is intracranial mouse recording, from a 16 channel microarray. The paper can be found here: https://doi.org/10.1371/journal.pone.0221510


# Getting Set Up

Open a terminal. Make sure "Sorter" environment is active. 

```
conda deactivate
conda activate sorter
```

navigate to the correct directory, and examine the folders available. 

```
cd ~/codespace
box folders:items 352606395707
```

Download the patient level folder, which will contain multiple session folders. 
Replace with the correct file number.
```
box folders:download 123456789 --destination="data"
```

In [26]:
import os
from pathlib import Path

codespace = Path.home() / "codespace"
base_folder = codespace / "data"
patient = "Intan_RDH_2000"
session = "Session1"
session_location =  base_folder / patient / session
sorted_data = session_location / "sorted"
sorter_output_folder = sorted_data / "sorter_folder" 
analyzer_folder = sorted_data / "analyzer_folder"

os.chdir(session_location)

intan_file = "/home/test/codespace/data/Intan_RDH_2000/Session1/raw/Intan RHD file1.rhd"

In [1]:
from pathlib import Path
import requests, zipfile, io

url = "https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/w767nnk5wh-1.zip"
base_folder = Path.cwd() / "Data"
zip_path = base_folder / "intan_data.zip"
extract_to = base_folder / "intan_data"


print("Downloading File...")
response = requests.get(url, stream=True)
response.raise_for_status()

with open(zip_path, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"Download complete: {zip_path}")

# 3. Unzip to a folder
print("Extracting files...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Files extracted to: {extract_to}")

Download complete: /home/ubuntu/codespace/sorter-script/Data/intan_data.zip
Extracting files...
Files extracted to: /home/ubuntu/codespace/sorter-script/Data/intan_data


# Load recording into spike interface

In [27]:
# Load Intan
from spikeinterface.sorters import run_sorter
import spikeinterface.full as si
import probeinterface as pi
from pathlib import Path

# Load Recording
rec = si.read_intan(intan_file, stream_id = "0")
rec

IntanRecordingExtractor: 16 channels - 20.0kHz - 1 segments - 24,000,480 samples 
                         1,200.02s (20.00 minutes) - uint16 dtype - 732.44 MiB
  file_path: /home/test/codespace/data/Intan_RDH_2000/Session1/raw/Intan RHD file1.rhd

In [28]:
# Create custom probe geometry
import probeinterface as pi

probe = pi.Probe(ndim=2)
positions = []

for i in range(16):
    positions.append([0, i * 50])
probe.set_contacts(positions = positions, shapes = "circle", shape_params = {'radius':5})

probe.set_device_channel_indices(range(16))
probe.set_contact_ids([f"ch{i}" for i in range(16)])

probe_path = codespace / "sorting_script/neuronexus-A16x1_2mm_50_177_A16.json"
pi.write_probeinterface(probe_path, probe)


In [29]:
# Attach Probe to Recording
rec = rec.set_probe(probe)

n_rec = rec.get_num_channels()
n_probe = probe.get_contact_count()

if n_probe != n_rec:
    raise ValueError(f"Probe contacts ({n_probe}) != recording channels ({n_rec}). "
                     f"Pick the correct probe variant or subset/remap accordingly.")

In [6]:
# Run Kilosort
from os import remove


sorting_KS4 = run_sorter(
    sorter_name="kilosort4",
    recording=rec,
    folder=sorter_output_folder,
    remove_existing_folder = True,
    verbose=True
)

write_binary_recording (no parallelization):   0%|          | 0/1201 [00:00<?, ?it/s]

kilosort.run_kilosort:  
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: N samples: 24000480
kilosort.run_kilosort: N seconds: 1200.024
kilosort.run_kilosort: N batches: 401
kilosort.run_kilosort: Preprocessing filters computed in 0.42s; total 0.42s
kilosort.run_kilosort:  
kilosort.run_kilosort: Resource usage after preprocessing
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort: CPU usage:     5.60 %
kilosort.run_kilosort: Mem used:      7.20 %     |       4.47 GB
kilosort.run_kilosort: Mem avail:    57.63 / 62.10 GB
kilosort.run_kilosort: ------------------------------------------------------
kilosort.run_kilosort: GPU usage:    `conda install pynvml` for GPU usage
kilosort.run_kilosort: GPU memory:    1.91 %     |      0.28   /    14.56 GB
kilosort.run_kilosort: Allocated:     0.06 %     |      0.01   /    14.56 GB
kilosort.run_kilosor

kilosort4 run time 34.71s


In [30]:
# Create Sorting Analyzer
import spikeinterface.full as si

# Load Recording
recording = si.read_intan(intan_file, stream_id = "0")
recording = recording.set_probe(probe, in_place=False)
recording = si.unsigned_to_signed(recording)
recording_filtered = si.bandpass_filter(recording)

job_kwargs = dict(n_jobs=-1, progress_bar=True, chunk_duration="1s")

sorting_analyzer = si.create_sorting_analyzer(sorting=sorting_KS4, recording=recording_filtered, folder=analyzer_folder, overwrite = True,
format="binary_folder", **job_kwargs)

sorting_analyzer.compute("random_spikes", method="uniform", max_spikes_per_unit=500)
sorting_analyzer.compute("waveforms", **job_kwargs)
sorting_analyzer.compute("templates", **job_kwargs)
sorting_analyzer.compute("noise_levels")
sorting_analyzer.compute("unit_locations", method = "monopolar_triangulation")
sorting_analyzer.compute("isi_histograms")
sorting_analyzer.compute("correlograms", window_ms=100, bin_ms=5)
sorting_analyzer.compute("principal_components", n_components=3, mode="by_channel_global", whiten=True, **job_kwargs)
sorting_analyzer.compute("quality_metrics", metric_names=["snr", "firing_rate"])
sorting_analyzer.compute("template_similarity")
sorting_analyzer.compute("spike_amplitudes", **job_kwargs)

estimate_sparsity (workers: 16 processes):   0%|          | 0/1201 [00:00<?, ?it/s]

/home/test/miniconda3/envs/sorter/lib/python3.11/site-packages/spikeinterface/core/basesorting.py:380: UserWarning: The registered recording will not be persistent on disk, but only available in memory
  warnings.warn("The registered recording will not be persistent on disk, but only available in memory")


compute_waveforms (workers: 16 processes):   0%|          | 0/1201 [00:00<?, ?it/s]

noise_level (no parallelization):   0%|          | 0/20 [00:00<?, ?it/s]

Fitting PCA:   0%|          | 0/21 [00:00<?, ?it/s]

Projecting waveforms:   0%|          | 0/21 [00:00<?, ?it/s]

spike_amplitudes (workers: 16 processes):   0%|          | 0/1201 [00:00<?, ?it/s]

In [31]:
analyzer_folder

PosixPath('/home/test/codespace/data/Intan_RDH_2000/Session1/sorted/analyzer_folder')

In [ ]:
# Run Curation GUI

# I need to figure out how to clear the cache in between runs so that the data will be cleared, without getting phantom lables. 

# The reason is because the web app panel caches the curation json, probably as a way to try and be helpful. It needs to be flushed in between sessions. 
import spikeinterface.full as si
from spikeinterface_gui import run_mainwindow

sorting_analyzer = si.load_sorting_analyzer(folder=analyzer_folder)

run_mainwindow(sorting_analyzer, mode="web", curation=True, start_app = True)

Found available port: 55063
Launching server at http://localhost:55063
